In [12]:
from tsl.datasets import MetrLA


dataset = MetrLA(root='/tsl/data')

print(dataset)

MetrLA(length=34272, n_nodes=207, n_channels=1)


In [13]:
import numpy as np
import pandas as pd
import random
random.seed(42)

In [14]:
distances = dataset.dist
print(distances.shape)
print(sum(distances))

(207, 207)
[inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf]


## TODO:

1. Make masks with random seed.
2. Save masks to csv_file
3. Save masked datasets 
4. push to git
5. Finish KNN implementation
6. Compute euclidean distance metrics from tsl/data/locations
7. Fill in text in report

##

In [15]:
print(np.reshape(distances, (-1, )).shape)
print(sum(np.isinf((np.reshape(distances, (-1, ))))))

(42849,)
31096


In [16]:
df = pd.DataFrame(dataset.dataframe())
# print(df.head())
df.info

<bound method DataFrame.info of nodes                   773869     767541     767542     717447     717446  \
channels                     0          0          0          0          0   
2012-03-01 00:00:00  64.375000  67.625000  67.125000  61.500000  66.875000   
2012-03-01 00:05:00  62.666668  68.555557  65.444443  62.444443  64.444443   
2012-03-01 00:10:00  64.000000  63.750000  60.000000  59.000000  66.500000   
2012-03-01 00:15:00  64.000000  63.750000  60.000000  59.000000  66.500000   
2012-03-01 00:20:00  64.000000  63.750000  60.000000  59.000000  66.500000   
...                        ...        ...        ...        ...        ...   
2012-06-27 23:35:00  65.000000  65.888885  68.555557  61.666668  32.833332   
2012-06-27 23:40:00  61.375000  65.625000  66.500000  62.750000  32.833332   
2012-06-27 23:45:00  67.000000  59.666668  69.555557  61.000000  32.833332   
2012-06-27 23:50:00  66.750000  62.250000  66.000000  59.625000  32.833332   
2012-06-27 23:55:00  65.111115  

In [17]:
print(df.shape)
print(df.columns)
df.columns = [str(tup[0]) for tup in df.columns]
print(df.columns)

(34272, 207)
MultiIndex([('773869', 0),
            ('767541', 0),
            ('767542', 0),
            ('717447', 0),
            ('717446', 0),
            ('717445', 0),
            ('773062', 0),
            ('767620', 0),
            ('737529', 0),
            ('717816', 0),
            ...
            ('772167', 0),
            ('769372', 0),
            ('774204', 0),
            ('769806', 0),
            ('717590', 0),
            ('717592', 0),
            ('717595', 0),
            ('772168', 0),
            ('718141', 0),
            ('769373', 0)],
           names=['nodes', 'channels'], length=207)
Index(['773869', '767541', '767542', '717447', '717446', '717445', '773062',
       '767620', '737529', '717816',
       ...
       '772167', '769372', '774204', '769806', '717590', '717592', '717595',
       '772168', '718141', '769373'],
      dtype='object', length=207)


In [18]:
df.index

DatetimeIndex(['2012-03-01 00:00:00', '2012-03-01 00:05:00',
               '2012-03-01 00:10:00', '2012-03-01 00:15:00',
               '2012-03-01 00:20:00', '2012-03-01 00:25:00',
               '2012-03-01 00:30:00', '2012-03-01 00:35:00',
               '2012-03-01 00:40:00', '2012-03-01 00:45:00',
               ...
               '2012-06-27 23:10:00', '2012-06-27 23:15:00',
               '2012-06-27 23:20:00', '2012-06-27 23:25:00',
               '2012-06-27 23:30:00', '2012-06-27 23:35:00',
               '2012-06-27 23:40:00', '2012-06-27 23:45:00',
               '2012-06-27 23:50:00', '2012-06-27 23:55:00'],
              dtype='datetime64[ns]', length=34272, freq='5T')

In [19]:
def create_random_mask(df, mask_percentage=20):
    total_elements = df.size
    mask_count = int(total_elements * mask_percentage / 100)
    mask_indices = np.random.choice(total_elements, mask_count, replace=False)
    mask = np.zeros(total_elements, dtype=bool)
    mask[mask_indices] = True
    mask = mask.reshape(df.shape)
    return mask

def create_sequential_mask(df, max_downtime, probability=0.001):
    mask = np.zeros(df.shape, dtype=bool)
    timesteps = df.shape[0]
    i = 0
    while i < timesteps:
        if np.random.rand() < probability:
            downtime = np.random.randint(0, max_downtime + 1)
            mask[i:min(i + downtime, timesteps), :] = True
            i += downtime
        else:
            i += 1
    return mask

In [20]:

max_downtime = 12*24 # Equivalent to 24 hours
random_mask = create_random_mask(df)
sequential_mask = create_sequential_mask(df, probability=0.002, max_downtime = max_downtime)
print(sequential_mask.shape)
num_timesteps = df.shape[0]
num_stations = df.shape[1]


(34272, 207)


In [21]:
random_masked_df = df.mask(random_mask)
print(type(random_masked_df))
sequential_masked_df = df.mask(sequential_mask)
nan_count_sequential = sequential_masked_df.isna().sum().sum()
count = random_masked_df.shape[0] * random_masked_df.shape[1]
nan_count_random = random_masked_df.isna().sum().sum()
print(f"Random mask: {nan_count_random} NaNs out of {count} ({nan_count_random/count*100:.2f}%)")
print(f"Sequential mask: {nan_count_sequential} NaNs out of {count} ({nan_count_sequential/count*100:.2f}%)")


<class 'pandas.core.frame.DataFrame'>
Random mask: 1418860 NaNs out of 7094304 (20.00%)
Sequential mask: 1984923 NaNs out of 7094304 (27.98%)


In [22]:
random_mask = pd.DataFrame(random_mask)
sequential_mask = pd.DataFrame(sequential_mask)
# print(random_mask)
dataframes_to_save = {'random_mask': random_mask, 'sequential_mask': sequential_mask, 'original_df': df, 'random_masked_df': random_masked_df, 'sequential_masked_df': sequential_masked_df}
folder_name = 'data'
import os
# Check if the folder exists, and if not, create it
# if not os.path.exists(folder_name):
#     os.makedirs(folder_name)
# for name, df in dataframes_to_save.items():
#     filename = f'{name}.csv'
#     df.to_csv(os.path.join(folder_name, filename), index=False)

In [23]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Define imputers
mean_imputer = SimpleImputer(strategy='mean')
median_imputer = SimpleImputer(strategy='median')
frequent_imputer = SimpleImputer(strategy='most_frequent')

# Apply imputation and calculate performance
performance_metrics = {}
for imputer, name in [(mean_imputer, 'Mean'), (median_imputer, 'Median'), (frequent_imputer, 'Most Frequent')]:
    imputed_data = imputer.fit_transform(random_masked_df)
    # Assuming 'original_data' is the original DataFrame without missing values
    mse = mean_squared_error(df, imputed_data)
    performance_metrics[name] = mse


In [ ]:
print(performance_metrics)

{'Mean': 26.610064, 'Median': 31.113073, 'Most Frequent': 47.990116}


In [ ]:
class KNNImpute():
    def __init__(self, k = 3, masked_df = None, distances = None):
        self.k = k
        self.masked_df = masked_df
        self.distances = distances
        self.stations = len(masked_df.columns)
        self.timesteps = len(masked_df.index)
        if not self.masked_df or not self.distances:
            raise Exception('masked_df and distances must be provided')
        self.knn_df = self.calulate_knn()
    
    def calulate_knn(self):
        nearest_neighbors_df = pd.DataFrame(index=df.index, columns=df.columns)

        for index, row in df.iterrows():
            # Sort the row while keeping track of the indices, and take the first k+1 indices (including the node itself)
            sorted_indices = row.argsort()[:k+1]
            
            # Exclude the node itself from its list of nearest neighbors
            nearest_neighbors = [idx for idx in sorted_indices if idx != index][:self.k]
            nearest_neighbors = [neighbor for neighbor in nearest_neighbors if not np.isnan(row[neighbor])]
            
            # Assign this list to the corresponding cell in the new DataFrame
            nearest_neighbors_df.loc[index] = nearest_neighbors

        return nearest_neighbors_df

    def get_knn(self, row, col):
        return self.knn_df.loc[row, col]
    
    def get_knn_df(self):
        return self.knn_df

    def fit(self, X):
        self.X = X
    
    def predict(self, X):
        pass

In [84]:
for index, row in df.iterrows():
    for col_name in df.columns:
        print(index, row, col_name)
        break
    break

2012-03-01 00:00:00 773869    64.375
767541    67.625
767542    67.125
717447    61.500
717446    66.875
           ...  
717592    59.375
717595    69.000
772168    59.250
718141    69.000
769373    61.875
Name: 2012-03-01 00:00:00, Length: 207, dtype: float32 773869
